<a href="https://colab.research.google.com/github/jstephencorey/bert-experiments/blob/main/bert_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
on_colab = True
first_run = False # Change this when you disconnect and reconnect the runtime
if on_colab and first_run:
    !git clone https://github.com/jstephencorey/bert-experiments.git  bert_git # Only need to run this on colab
    from pathlib import Path
    import sys
    sys.path.append(str('/content/bert_git')) #Only need on the colab
    print(sys.path)
    !pip install transformers --quiet
    !pip install wandb --quiet

In [8]:
import torch
from torch import nn
import random
from torch import optim
import bert
import wandb
wandb.login()

wandb: Currently logged in as: jstephencorey (sintez). Use `wandb login --relogin` to force relogin


True

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Import the tokenizers

In [10]:
from transformers import AutoTokenizer

In [11]:
DATA_DIR = '/content/drive/MyDrive/AI_Data/'
MODEL_DIR = '/content/drive/MyDrive/AI_Models/storygpt/'
TOKENIZER_DIR = f'{MODEL_DIR}storygpt2tokenizer_ft2/'

bibliotik_train = f'{DATA_DIR}bibliotik_corpus/biblitik_22500_full.gz'
bibliotik_val = f'{DATA_DIR}bibliotik_corpus/biblitik_7500_val.gz'

atlas_shrugged_filename = f'{DATA_DIR}atlas-shrugged.txt'
anthem_filename = f'{DATA_DIR}anthem.txt'

#Set up the model

In [4]:
test_parts = True
device= "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
run_name = "bert_init_test_0"

wandb.init(project="bert_experiments", 
           name=run_name,
           )

wandb.config = {
  "epochs": 5,
  "batch_size": None,
  "val_batch_size": None,
  "seed":135,
  # Model info
  "context_len": 256,
  "d_model":768,
  "num_layers":12,
  "n_head":12,
  "lr": 1e-4,
  # "stories_per_epoch": 5000,
  "save_model": True,
  "from_pretrained": False,#f'{MODEL_DIR}gpt-3-test-8-0.pt'
  "val_seqs_per_story": 32, #More is faster, but smaller is more generalizable
  "bert_model_name": 'distilbert-base-cased',
  "bert_mask_percentage": .2,
}

config = wandb.config

random.seed(config['seed'])
torch.random.manual_seed(config['seed'])

print(config)

In [5]:
model = bert.BertModel(d_model = 512, 
                        vocab_length = 30, 
                        sequence_length = config['context_len'],
                        num_layers = 8, 
                        feed_forward_dimensions = 1024, 
                        attention_heads = 8,
                        attention_qkv_dims =  128, 
                        dropout = 0.1, 
                        pad_idx = 3, 
                        device = "CPU")

In [6]:
if test_parts:
    inp = torch.LongTensor([[0,1,2,3,4,5,6,7,8,29],[1,3,4,5,6,8,0,22,1,1]])
    out = model(inp)
    print(out.size())

torch.Size([2, 10, 30])


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
batch = torch.LongTensor([[0,1,2,3,4,5,6,7,8,29],[1,3,4,5,6,8,0,22,1,1]])
print(batch.size())
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(config["epochs"]):
    optimizer.zero_grad()
    logits_lm = model(batch)
    print(logits_lm.size(), batch.size())
    loss_lm = criterion(logits_lm.transpose(1, 2), batch) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    # loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm #+ loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

# # Predict mask tokens
# input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
# print(text)
# print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

# logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
# logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
# print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
# print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

# logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
# print('isNext : ', True if isNext else False)
# print('predict isNext : ',True if logits_clsf else False)

torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
torch.Size([2, 10, 30]) torch.Size([2, 10])
Epoch: 0010 cost = 53.527332
